This notebook serves as template for the calculation of the results presented in the paper "*PULPO🐙: A framework for efficient integration of life cycle inventory models into life cycle product optimization*".

In this specific notebook, one optimization is performed based on the users specification of (scenario/year/electricity bound)

### Import Section

In [ ]:
import os
import sys
sys.path.append('../pulpo/')
from pulpo import pulpo
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Setup

In [ ]:
# Specify the project and database names as well as the impact to minimize
project = "pulpo_example"
scenario = "PkBudg500"
year = "2040"
database = "CCU_" + scenario + "_" + year
method = "('IPCC 2013', 'climate change', 'GWP 100a, incl. H and bio CO2')"

# Substitute with your working directory of choice
notebook_dir = os.getcwd()
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
GAMS_PATH = "C:/GAMS/37/gams.exe"

Create a PULPO worker

In [ ]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, method, directory)

Retrieve the data (*from brightway2*). If data is already loaded, this step is automatically skipped. 

In [ ]:
pulpo_worker.get_lci_data()

### Specify FU

In [ ]:
reference_products=["electricity, high voltage"] # 💡
activities = ["market group for electricity, high voltage"] # 📊
locations = ["CAZ","USA","NEU","SSA","EUR","IND","LAM","OAS","REF","MEA","CHA","JPN"] # 🌎

In [ ]:
elec_markets = pulpo_worker.retrieve_activities(reference_products=reference_products, activities=activities, locations=locations)

In [ ]:
meoh_market = pulpo_worker.retrieve_activities(activities=["new market for methanol"]) # 🧪📊

*IMPORTANT*: This step requires as an input decrypted IAM output files. The encrypted files are storred within the premise package. To decrypt them, the same key as for the premise installation is required. The notebook "decrypt_iam_output.ipynb" shows how to do that.

In [ ]:
notebook_dir = os.getcwd()
input_path = os.path.join(notebook_dir, 'data\\raw\\decrypted\\')
iam_output = pd.read_csv(input_path + 'decrypted_remind_SSP2-' + scenario + '.csv')
filtered_df = iam_output[iam_output['Variable'] == 'SE|Electricity'].loc[:, ['Region', year]]

In [ ]:
elec_supply = {filtered_df['Region'][ind]: filtered_df[year][ind]/3.6E-12 for ind in filtered_df.index if filtered_df['Region'][ind] != 'World'}

In [ ]:
elec_supply

In [ ]:
demand = {meoh_market[0]: 100e9}

### Choices

**Methanol 🧪 production**, retrieve all the synthesis processes, using DAC and PSC

In [ ]:
activities = ["methanol synthesis, hydrogen from electrolysis, CO2 from DAC", "methanol synthesis, hydrogen from electrolysis, CO2 from point-source"]
locations = ["CAZ","USA","NEU","SSA","EUR","IND","LAM","OAS","REF","MEA","CHA","JPN"]
meoh_choices = pulpo_worker.retrieve_activities(activities=activities, locations=locations)
meoh_choices.append(pulpo_worker.retrieve_activities(activities=["methanol production"], locations=["GLO"])[0])
meoh_choices.append(pulpo_worker.retrieve_activities(reference_products=["methanol"], activities=["synthetic fuel production, from coal, high temperature Fisher-Tropsch operations"], locations=["ZA"])[0])

**Electricity 💡 retrofitting**, for selected fossil thermal powerplants

In [ ]:
pp_name = ['electricity production, hard coal',
           'electricity production, hard coal, conventional',
           'electricity production, hard coal, supercritical',
           'heat and power co-generation, hard coal',
           'electricity production, lignite',
           'heat and power co-generation, lignite',
           'electricity production, natural gas, conventional power plant',
           'electricity production, natural gas, combined cycle power plant',
           'heat and power co-generation, natural gas, conventional power plant, 100MW electrical',
           'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical']
pp_name = [name + ', retrofitting with CC' for name in pp_name]

In [ ]:
# Retrieve all retrofitting activities and create a searchable list
retrofit_activities = pulpo_worker.retrieve_activities(activities=pp_name)
retrofit_activities_list = [[act['name'], act['location'], act['reference product']] for act in retrofit_activities]

In [ ]:
# Retrieve all the activities that have the same names as the retrofitting ones, but without CC
act_orig_names = [act['name'].replace(', retrofitting with CC','') for act in retrofit_activities]
act_orig_raw = pulpo_worker.retrieve_activities(activities=act_orig_names)

In [ ]:
# Filter that list to only include the correct types of plant in the correct region
original_activities = [act for act in act_orig_raw if [act['name']+', retrofitting with CC', act['location'], act['reference product']] in retrofit_activities_list]

In [ ]:
# Match the non-CC and the CC activities in one dictionary
n= 0
retrofit_choices = {}
for act in retrofit_activities: 
    j = 0
    while [act['name'], act['location'], act['reference product']] != [original_activities[j]['name']+', retrofitting with CC', original_activities[j]['location'], original_activities[j]['reference product']]:     
            j+=1
    retrofit_choices[str(n)] = [act, original_activities[j]]
    n+=1

In [ ]:
choices  = {n: {retrofit_choices[n][0]: 1e20,
                retrofit_choices[n][1]: 1e20} for n in retrofit_choices}
choices['methanol'] = {act: 1e20 for act in meoh_choices}

### Constraints

In [ ]:
elec_bound = 0

In [ ]:
upper_bound = {elec_market: elec_supply[elec_market['location']]*(1+elec_bound) for elec_market in elec_markets}
lower_bound = {elec_market: elec_supply[elec_market['location']] for elec_market in elec_markets}

### Instantiate and Solve (BASE CASE)

Prior to solving the optimization problem involving the choices and the electricity capacity extension, the reference situation (BASE CASE) must be calculated. This case corresponds to the situation where **exactly** the forecasted electricity is supplied (via the PULPO functionality of fixing a supply), as well as the demand for methanol.

##### only methanol 🧪:

In [ ]:
pulpo_worker.instantiate(choices={}, demand=demand, upper_limit=upper_bound)

In [ ]:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

In [ ]:
reference_meoh_GWP = pulpo_worker.instance.OBJ()
pulpo_worker.save_results(choices={}, demand=demand, name='meoh_only_meoh.xlsx')

##### methanol 🧪 and elec 💡:

In [ ]:
pulpo_worker.instantiate(choices={}, demand=demand, lower_limit=lower_bound, upper_limit=upper_bound)

In [ ]:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

In [ ]:
pulpo_worker.save_results(choices={}, demand=demand, name='meoh_with_elec.xlsx')

In [ ]:
reference_GWP = pulpo_worker.instance.OBJ()

In [ ]:
try:
    reference_elec = sum(pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][x]].value for x in elec_markets)
except: 
    reference_elec = sum(pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][x]] for x in elec_markets)

This result serves as a reference for the subsequent optimization. Moreover, the slack variables for the electricity supply are used as the new final demand. 

### Instantiate and Solve (WITH CHOICES 🕸️)

First, specify the percentual increase in electricity that is to be evaluated:

In [ ]:
elec_bound = 0.1

Specify the final demand of electricity calculated from the BASE CASE, as well as the methanol demand.

In [ ]:
try:
    final_electricity_demand = {x: pulpo_worker.instance.slack[x].value for x in pulpo_worker.instance.slack}
except:
    final_electricity_demand = {x: pulpo_worker.instance.slack[x] for x in pulpo_worker.instance.slack}
demand = {x: final_electricity_demand[pulpo_worker.lci_data['process_map'][x]] for x in elec_markets}
demand[meoh_market[0]] = 100000000000

Specifiy that the CO2 markets may not take on negative values as this would "destroy" CO2.

In [ ]:
carbon_markets = pulpo_worker.retrieve_activities(activities=["carbon dioxide, captured from point-source"])
lower_bound_carbon = {carbon_market: 0 for carbon_market in carbon_markets}

Specify the lower and upper bounds on the electricity supply

In [ ]:
upper_bound = {elec_market: elec_supply[elec_market['location']]*(1+elec_bound) for elec_market in elec_markets}
lower_bound = {elec_market: elec_supply[elec_market['location']] for elec_market in elec_markets}
lower_bound.update(lower_bound_carbon)

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand, lower_limit=lower_bound, upper_limit=upper_bound)

In [ ]:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

In [ ]:
pulpo_worker.save_results(choices={}, demand=demand, name='meoh_results.xlsx')

Calculate the GWP reduction of the optimized case with the base case:

In [ ]:
optimal_GWP = pulpo_worker.instance.OBJ()

In [ ]:
gwp_reduction = (reference_GWP-optimal_GWP)*0.000000001

In [ ]:
print(str(round((optimal_GWP-reference_GWP)*0.000000001,2)) + ' MT')

### Save the results 📊

In [ ]:
try:
    optimized_elec = sum(pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][x]].value for x in elec_markets)
except:
    optimized_elec = sum(pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][x]] for x in elec_markets)

Create dataframe with proper information:

In [ ]:
meoh_production = {}
location_mapping = {
    'GLO': ('methanol production', 'BAU'),
    'ZA': ('Fischer Tropsch', '-'),
}

i = 0
for choice in choices['methanol']:
    region = choice['location']
    
    if region == 'GLO':
        region, tech = location_mapping[region]
        elec = (optimized_elec - reference_elec) / 1e9
        net_zero = reference_meoh_GWP / 1e9
        gwp = gwp_reduction
    elif 'DAC' in choice['name']:
        tech = 'DAC'
        elec, net_zero, gwp = 0, 0, 0
    elif 'point-source' in choice['name']:
        tech = 'PSC' 
        elec, net_zero, gwp = 0, 0, 0
    if region != 'ZA':
        try:
            value = pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][choice]].value * 0.000000001
        except:
            value = pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][choice]] * 0.000000001
        meoh_production[i] = {
            'scenario': scenario,
            'year': int(year),
            'epsilon [%]': str(round(elec_bound*100,3)),
            'region': region,
            'tech': tech,
            'meoh [Mt]': value,
            'elec [TWh]': elec,
            'net-zero GWP [Mt]': net_zero,
            'GWP reduction [Mt]': gwp
        }
        i += 1

Function for appending the dataframe to the already calculated results:

In [ ]:
def save_to_excel(dataframe, file_path):
    if not os.path.isfile(file_path):
        # If the Excel file doesn't exist, create a new one
        dataframe.to_excel(file_path, index=False)
    else:
        # If the Excel file already exists, check if the data is already present
        existing_data = pd.read_excel(file_path)
        existing_combinations = set(zip(existing_data["scenario"], existing_data["year"], existing_data["epsilon [%]"]))
        new_combinations = set(zip(dataframe["scenario"], dataframe["year"], dataframe["epsilon [%]"]))
        
        if not any(combination in existing_combinations for combination in new_combinations):
            # Append the new data to the existing Excel file
            with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                pd.concat([existing_data, dataframe], ignore_index=True).to_excel(writer, index=False, sheet_name='Sheet1')

Save results:

In [ ]:
data = pd.DataFrame(meoh_production).transpose()

In [ ]:
# File path where you want to save the results
file_path = directory + "\\results\\results_meoh_case.xlsx"
# Call the function to save the DataFrame
save_to_excel(data, file_path)

### Loop

For calculating the values for Figure 3

In [ ]:
import numpy as np

In [ ]:
for bound in np.linspace(0.0005,0.025,50):
    bound = round(bound,4)
    print(bound)
    upper_bound = {elec_market: elec_supply[elec_market['location']]*(1+bound) for elec_market in elec_markets}
    pulpo_worker.instantiate(choices=choices, demand=demand, lower_limit=lower_bound, upper_limit=upper_bound)
    results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)
    optimal_GWP = pulpo_worker.instance.OBJ()
    gwp_reduction = (reference_GWP-optimal_GWP)/1e9
    try:
        optimized_elec = sum(pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][x]].value for x in elec_markets)
    except:
        optimized_elec = sum(pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][x]] for x in elec_markets)
    meoh_production = {}
    location_mapping = {
        'GLO': ('methanol production', 'BAU'),
        'ZA': ('Fischer Tropsch', '-'),
    }

    i = 0
    for choice in choices['methanol']:
        region = choice['location']

        if region == 'GLO':
            region, tech = location_mapping[region]
            elec = (optimized_elec - reference_elec) / 1e9
            net_zero = reference_meoh_GWP / 1e9
            gwp = gwp_reduction
        elif 'DAC' in choice['name']:
            tech = 'DAC'
            elec, net_zero, gwp = 0, 0, 0
        elif 'point-source' in choice['name']:
            tech = 'PSC' 
            elec, net_zero, gwp = 0, 0, 0
        if region != 'ZA':
            try:
                value = pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][choice]].value * 0.000000001
            except:
                value = pulpo_worker.instance.scaling_vector[pulpo_worker.lci_data['process_map'][choice]] * 0.000000001
            meoh_production[i] = {
                'scenario': scenario,
                'year': int(year),
                'epsilon [%]': str(round(bound*100,3)),
                'region': region,
                'tech': tech,
                'meoh [Mt]': value,
                'elec [TWh]': elec,
                'net-zero GWP [Mt]': net_zero,
                'GWP reduction [Mt]': gwp
            }
            i += 1
    data = pd.DataFrame(meoh_production).transpose()
    save_to_excel(data, file_path)